# STL to block mesh dictionary python script 
### 1/9/23 from stl_bmd.ipynb


In [62]:
from __future__ import unicode_literals, print_function

import math
import numpy as np
from ofblockmeshdicthelper import BlockMeshDict, Vertex, SimpleGrading
from collections import OrderedDict


In [63]:
# ^ y
# |  /z (to the outlet )
# | /
# |/_________> x    


# ^ y
# |
# |
# |__________________________> z

In [64]:
# take in box parameters and build a mesh around them 
box_params = { #m
    "x": 4.65, # x-y plane faces front of the wind tunnel, width 
    "z": 3.65, # depth of the box 
    "y": 3.66 # height 0 vertical 
}



In [65]:
## wind tunnel params 
# ramponi + blocken 2012        
# * H_D = H + d, 
# * W_D = W + 2d
# * H, W = height and width of the building respectively 
# * d = 5H - 3H (start with 3H for early study) 
# * Upstream, downstream length => 5H and 15H 

H = box_params["y"]
d = 3 * H # 3H 

wt_params = {
    "x": box_params["x"] + 2*d,   # width / length
    "z": box_params["z"] + 5*H + 15*H, # depth 
    "y": box_params["y"] + d, # height 
}
wt_params = OrderedDict(wt_params)

In [66]:
# smallest dimension of the building to find the minimum grid size 
min_dim = min(box_params.values())
min_grid_size = min_dim/10
min_grid_size # 0.35 .... maybe consider the windows for this?

# grid size in areas of interest, again what about winodws?
box_volume = np.prod(list(box_params.values()))
box_volume_croot = box_volume**(1/3)
interest_grid_size = box_volume_croot/10

# away from areas of interest 
n = 1
expansion_ratio = 1.3
uninterest_grid_size = interest_grid_size * (1.3**n)

# determine number of cells in each direction 
cells = [int(np.ceil(i/uninterest_grid_size)) for i in wt_params.values()]
cells = tuple(cells)
cells

(52, 150, 29)

## make block mesh dict 

In [67]:
# prepare ofblockmeshdicthelper.BlockMeshDict instance to
# gather vertices, blocks, faces and boundaries.
bmd = BlockMeshDict()

# set metrics
bmd.set_metric('m')


In [68]:
# wind tunnel 

basevs = [
    Vertex(0, 0, 0, 'v0'),
    Vertex(wt_params["x"], 0, 0, 'v1'),
    Vertex(wt_params["x"],  0, wt_params["z"], 'v2'),
    Vertex(0, 0, wt_params["z"], 'v3')]

# for v in basevs:
#     print(f"{v.name}: {v.x, v.y, v.z}")


# raise up in the y direction 
for v in basevs:
    bmd.add_vertex(v.x, wt_params["y"], v.z, v.name+'+y')
    bmd.add_vertex(v.x, 0, v.z, v.name+'-y')

for v in basevs:
    print(f"{v.name}: {v.x, v.y, v.z}")

v0: (0, 0, 0)
v1: (26.61, 0, 0)
v2: (26.61, 0, 76.85000000000001)
v3: (0, 0, 76.85000000000001)


In [69]:
# print names and coordinates of a block mesh dict object 
def print_vertices(bmd):
    for k,v in bmd.vertices.items():
        print(f"{k} : ({bmd.vertices[k].x}, {bmd.vertices[k].y}, {bmd.vertices[k].z})")

In [70]:
def vnamegen(x0z0, x1z0, x1z1, x0z1):
    return (x0z0+'-y', x1z0+'-y', x1z0+'+y', x0z0+'+y',
            x0z1+'-y', x1z1+'-y', x1z1+'+y', x0z1+'+y')

In [71]:
vnamegen('v0', 'v1', 'v2', 'v3')

('v0-y', 'v1-y', 'v1+y', 'v0+y', 'v3-y', 'v2-y', 'v2+y', 'v3+y')

In [72]:
b0 = bmd.add_hexblock(vnamegen('v0', 'v1', 'v2', 'v3'),
                      cells,
                      'b0',
                      grading=SimpleGrading(1,
                                            1,
                                            1))

In [73]:
        # index is number or keyword to identify the face of Hex
        #     0 = 'w' = 'xm' = '-100' = (0 4 7 3)
        #     1 = 'e' = 'xp' = '100' = (1 2 5 6)
        #     2 = 's' = 'ym' = '0-10' = (0 1 5 4)
        #     3 = 'n' = 'yp' = '010' = (2 3 7 6)
        #     4 = 'b' = 'zm' = '00-1' = (0 3 2 1)
        #     5 = 't' = zp' = '001' = (4 5 6 7)
        # name is given to Face instance. If omitt

In [74]:
print_vertices(bmd)

v0+y : (0, 14.64, 0)
v0-y : (0, 0, 0)
v1+y : (26.61, 14.64, 0)
v1-y : (26.61, 0, 0)
v2+y : (26.61, 14.64, 76.85000000000001)
v2-y : (26.61, 0, 76.85000000000001)
v3+y : (0, 14.64, 76.85000000000001)
v3-y : (0, 0, 76.85000000000001)


In [75]:
for i in range(6):
    print(b0.face(i).vnames)

('v0-y', 'v3-y', 'v3+y', 'v0+y')
('v1-y', 'v1+y', 'v2+y', 'v2-y')
('v0-y', 'v1-y', 'v2-y', 'v3-y')
('v1+y', 'v0+y', 'v3+y', 'v2+y')
('v0-y', 'v0+y', 'v1+y', 'v1-y')
('v3-y', 'v2-y', 'v2+y', 'v3+y')


In [76]:
boundaries = ["symmetry", "symmetry", "wall", "patch", "patch", "patch"]
names = ["side01", "side02", "bottom", "top", "inlet", "outlet"]
for i, boundary, name in zip(range(6), boundaries, names):
    bmd.add_boundary(boundary, name, [b0.face(i)])

In [77]:
for k,v in bmd.boundaries.items():
    print(f"{k}: {bmd.boundaries[k].faces[0].vnames, }")
    print(f"{k}: {bmd.boundaries[k].faces[0].name }")
    

side01: (('v0-y', 'v3-y', 'v3+y', 'v0+y'),)
side01: f-b0-w
side02: (('v1-y', 'v1+y', 'v2+y', 'v2-y'),)
side02: f-b0-n
bottom: (('v0-y', 'v1-y', 'v2-y', 'v3-y'),)
bottom: f-b0-s
top: (('v1+y', 'v0+y', 'v3+y', 'v2+y'),)
top: f-b0-n
inlet: (('v0-y', 'v0+y', 'v1+y', 'v1-y'),)
inlet: f-b0-b
outlet: (('v3-y', 'v2-y', 'v2+y', 'v3+y'),)
outlet: f-b0-t


# Finish 

In [78]:
bmd.assign_vertexid()

In [79]:
print(bmd.format())

/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  2.3.0                                 |
|   \\  /    A nd           | Web:      www.OpenFOAM.org                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    object      blockMeshDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

convertToMeters 1;

vertices
(
    (                  0                  0                  0 )  // 0 v0-y
    (              26.61                  0                  0 )  // 1 v1-y
    (              26.61              14.64                 

In [80]:
# automatically write to block mesh file? 